## Playground 🤟🏼

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2024-07-07 08:07:24.194284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 08:07:24.194417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 08:07:24.325426: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Loading the Dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist
for sets in [X_train, y_train, X_test, y_test]:
    print(sets.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# Creating the Validation Set
X_train, X_valid = X_train[:-8000], X_train[-8000:]
y_train, y_valid = y_train[:-8000], y_train[-8000:]

for sets in [X_train, y_train, X_valid, y_valid, X_test, y_test]:
    print(sets.shape)

(52000, 28, 28)
(52000,)
(8000, 28, 28)
(8000,)
(10000, 28, 28)
(10000,)


In [4]:
# Creating a Custom Loss function
class Accuracy(tf.keras.losses.Loss):
    
    # Leveraging the Parent Class properties
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    # Loss function
    def call(self, y_true, y_pred):
        acc = tf.reduce_sum(tf.cast(y_true == y_pred, tf.int32)) / y_true.shape[0]
        return acc

In [5]:
tf.keras.backend.clear_session()

simple_loss_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=[28, 28, 1], activation="elu"),
    tf.keras.layers.Conv2D(32, (3, 3), activation="elu"),
    tf.keras.layers.MaxPool2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation="elu"),
    tf.keras.layers.Conv2D(64, (3, 3), activation="elu"),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(512, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(256, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(10, activation="softmax")
])

simple_loss_model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,559,210 (9.76 MB)

 Trainable params: 2,559,210 (9.76 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
exponential_sch = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10e2,
    decay_rate=0.1,
    staircase=False
)

# Using the custom loss function for compilation
simple_loss_model.compile(
    loss=Accuracy(),
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Nadam(learning_rate=exponential_sch)
)

In [7]:
# Normalising and Standardising the Images
X_train_mean = X_train.mean()
X_train_std = X_train.std()

for sets in [X_train, X_valid, X_test]:
    sets = (sets - X_train_mean) / X_train_std
    sets = sets / 255

In [8]:
# # Early Stopping
# earlystop_cb = tf.keras.callbacks.EarlyStopping(
#     patience=7, restore_best_weights=True, mode="min", monitor="val_loss", min_delta=1e-3
# )

# # Training using the Custom Loss
# simple_loss_model.fit(
#     X_train, y_train, validation_data=(X_valid, y_valid), epochs=30, callbacks=[earlystop_cb]
# )